In [4]:
import csv
import time  # Just to compare fit times
from pathlib import Path
from pprint import pprint

In [16]:
import numpy as np
import ray
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tune_sklearn import TuneSearchCV
from xgboost import XGBRegressor
import pandas as pd

In [3]:
%load_ext memory_profiler

In [44]:
!~/.local/bin/ray start --head --port=6379 --num-cpus=40

Local node IP: 10.6.54.24
2021-01-20 15:03:20,238	INFO services.py:1092 -- View the Ray dashboard at http://localhost:8265

--------------------
Ray runtime started.
--------------------

Next steps
  To connect to this Ray runtime from another node, run
    ray start --address='10.6.54.24:6379' --redis-password='5241590000000000'
  
  Alternatively, use the following Python code:
    import ray
    ray.init(address='auto', _redis_password='5241590000000000')
  
  If connection fails, check your firewall settings and network configuration.
  
  To terminate the Ray runtime, run
    ray stop


In [7]:
# !~/.local/bin/ray --version

ray, version 1.0.1.post1


In [6]:
# ray.__version__

'1.0.1.post1'

In [43]:
!~/.local/bin/ray stop

Did not find any active Ray processes.


In [47]:
ray.init(address='auto', _redis_password='5241590000000000')

2021-01-20 15:03:29,523	INFO worker.py:651 -- Connecting to existing Ray cluster at address: 10.6.54.24:6379


{'node_ip_address': '10.6.54.24',
 'raylet_ip_address': '10.6.54.24',
 'redis_address': '10.6.54.24:6379',
 'object_store_address': '/tmp/ray/session_2021-01-20_15-03-19_741961_43497/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-20_15-03-19_741961_43497/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2021-01-20_15-03-19_741961_43497',
 'metrics_export_port': 63888,
 'node_id': 'd3b7c44a1054120eddd1d4322b200836fa7fefc7'}

In [46]:
ray.shutdown()

In [48]:
# read the csv file created
input_dataset = Path("../data/quantile_df.csv")
df = pd.read_csv(input_dataset)
df = df.drop(["x","y"],axis=1)


y = df['target'] 
X = df.drop("target",axis=1)

# X, y = train[:, 1:], train[:, :1]
# y = y.ravel()

# Set training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
# XGBoost
parameters = {
    "eval_metric": ['logloss', "rmse"],
    "n_estimators": [10,100],
    "max_depth": [5, 30],
    "learning_rate": [0.1, 10]
}

xgb_tune_search = TuneSearchCV(
    XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, alpha=10),
    parameters,

    search_optimization="bayesian",
#     max_iters=10,
    n_jobs=40,
    n_trials=200,
    early_stopping=True,

    verbose=1,
    return_train_score=True,
    loggers=["csv"],

)

start = time.time()
xgb_tune_search.fit(X_train, y_train)
end = time.time()
print("Tune Fit Time:", end - start)
y_pred = xgb_tune_search.predict(X_test)


Trial name,status,loc,eval_metric,learning_rate,max_depth,n_estimators
_Trainable_c602613c,RUNNING,,logloss,6.09683,23,50
_Trainable_c6266eba,RUNNING,,logloss,0.508988,18,55
_Trainable_c649ab00,RUNNING,,rmse,2.01223,18,47
_Trainable_c6620e7a,RUNNING,,rmse,9.06254,7,47
_Trainable_c6774556,RUNNING,,logloss,9.03975,24,29
_Trainable_c6869dee,RUNNING,,logloss,4.08791,19,89
_Trainable_c69c1598,RUNNING,,rmse,9.30218,16,94
_Trainable_c6af3542,RUNNING,,rmse,2.63628,12,55
_Trainable_c6c005ca,RUNNING,,rmse,4.50979,10,92
_Trainable_c6ce362c,RUNNING,,rmse,7.87707,30,16


2021-01-20 15:09:44,925	ERROR worker.py:1037 -- Possible unhandled error from worker: ray::_Trainable.train() (pid=43920, ip=10.6.54.24)
  File "python/ray/_raylet.pyx", line 482, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 436, in ray._raylet.execute_task.function_executor
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/ray/tune/trainable.py", line 336, in train
    result = self.step()
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 106, in step
    return self._train()
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 205, in _train
    for name, score in self.scoring.items()
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 205, in <dictcomp>
    for name, score in self.scoring.items()
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line

In [ ]:
xgb_tune_search.best_estimator_

In [ ]:
#save results
import pickle

fileObj = open('best_estimator_.pkl', 'wb')
pickle.dump(xgb_tune_search.best_estimator_,fileObj)
fileObj.close()

In [ ]:
#save results
import pickle

fileObj = open('xgb_tune_search.pkl', 'wb')
pickle.dump(xgb_tune_search,fileObj)
fileObj.close()

In [ ]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)

    return {'r2': round(r2, 4), 'MAE': round(mean_absolute_error, 4), 'MSE': round(mse, 4), 'RMSE': round(np.sqrt(mse), 4), "explained_variance": round(explained_variance, 4)}

In [ ]:
regression_results(y_test, y_pred)

## Bayesian Hyperparameter Optimization and XGBoost

In [22]:


###############################################
# Import Machine Learning Assets
###############################################
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

###############################################
# Import Miscellaneous Assets
###############################################
import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from functools import partial
from pprint import pprint as pp
from tqdm import tqdm, tqdm_notebook

pd.set_option('display.expand_frame_repr', False)

###############################################
# Declare Global Variables
###############################################
CROSS_VALIDATION_PARAMS = dict(n_splits=5, shuffle=True, random_state=32)
XGBOOST_REGRESSOR_PARAMS = dict(
    learning_rate=0.2, n_estimators=200, subsample=0.8, colsample_bytree=0.8, 
    max_depth=10, n_jobs=-1
)

BAYESIAN_OPTIMIZATION_MAXIMIZE_PARAMS = dict(
    init_points=1,  # init_points=20,
    n_iter=2,  # n_iter=60,
    acq='poi', xi=0.0
)
BAYESIAN_OPTIMIZATION_BOUNDARIES = dict(
    max_depth=(5, 12.99),
    gamma=(0.01, 5),
    min_child_weight=(0, 6),
    scale_pos_weight=(1.2, 5),
    reg_alpha=(4.0, 10.0),
    reg_lambda=(1.0, 10.0),
    max_delta_step=(0, 5),
    subsample=(0.5, 1.0),
    colsample_bytree=(0.3, 1.0),
    learning_rate=(0.0, 1.0)
)
BAYESIAN_OPTIMIZATION_INITIAL_SEARCH_POINTS = dict(
    max_depth=[5, 10],
    gamma=[0.1511, 3.8463],
    min_child_weight=[2.4073, 4.9954],
    scale_pos_weight=[2.2281, 4.0345],
    reg_alpha=[8.0702, 9.0573],
    reg_lambda=[2.0126, 3.5934],
    max_delta_step=[1, 2],
    subsample=[0.8, 0.8234],
    colsample_bytree=[0.8, 0.7903],
    learning_rate=[0.2, 0.1]
)

reserve_features = [
    'rs1_x', 'rs1_y', 'rs2_x', 'rs2_y', 'rv1_x', 'rv1_y', 'rv2_x', 'rv2_y',
    'total_reserve_dt_diff_mean', 'total_reserve_mean', 'total_reserve_sum'
]

BASE_ESTIMATOR = partial(XGBRegressor)
# train_df, test_df = None, None
# oof_predictions, test_predictions = None, None
# train_input = None
# best_round = None
# target = None



ERROR! Session/line number was not unique in database. History logging moved to new session 187


In [23]:
# read the csv file created
input_dataset = Path("../data/quantile_df.csv")
df = pd.read_csv(input_dataset)
df = df.drop(["x","y"],axis=1)


y = df['target'] 
X = df.drop("target",axis=1)

# train_df, test_df = None, NOne
# oof_predictions, test_predictions = None, None
# train_input = None
# best_round = None
# target = None

# X, y = train[:, 1:], train[:, :1]
# y = y.ravel()

# Set training and validation sets
train_df, test_df, target, test_predictions = train_test_split(X, y, test_size=0.33)

In [24]:


def search_node(**kwargs):
    # global train_df, test_df, train_input, oof, test_predictions, best_round, target
    global train_df, target

    ###############################################
    # Unify Parameters
    ###############################################
    received_params = dict(dict(n_estimators=200,), **{_k: _v if _k not in ('max_depth') else int(_v) for _k, _v in kwargs.items()})
    current_params = dict(XGBOOST_REGRESSOR_PARAMS, **received_params)

    ###############################################
    # Initialize Folds and Result Placeholders
    ###############################################
    folds = KFold(**CROSS_VALIDATION_PARAMS)
    evaluation = np.zeros((current_params['n_estimators'], CROSS_VALIDATION_PARAMS['n_splits']))
    oof_predictions = np.empty(len(train_df))
    np.random.seed(32)

    progress_bar = tqdm_notebook(
        enumerate(folds.split(target, target)), 
        total=CROSS_VALIDATION_PARAMS['n_splits'], 
        leave=False
    )
    
    ###############################################
    # Begin Cross-Validation
    ###############################################
    for fold, (train_index, validation_index) in progress_bar:
        train_input, validation_input = train_df.iloc[train_index], train_df.iloc[validation_index]
        train_target, validation_target = target.iloc[train_index], target.iloc[validation_index]

        ###############################################
        # Initialize and Fit Model With Current Parameters
        ###############################################
        model = BASE_ESTIMATOR(**current_params)
        eval_set = [(train_input, train_target), (validation_input, validation_target)]
        model.fit(train_input, train_target, eval_set=eval_set, verbose=False)

        ###############################################
        # Find Best Round for Validation Set
        ###############################################
        evaluation[:, fold] = model.evals_result_["validation_1"]['rmse']
        best_round = np.argsort(evaluation[:, fold])[0]

        progress_bar.set_description('Fold #{}:   {:.5f}'.format(
            fold, evaluation[:, fold][best_round]
        ), refresh=True)

    ###############################################
    # Compute Mean and Standard Deviation of RMSLE
    ###############################################
    mean_eval, std_eval = np.mean(evaluation, axis=1), np.std(evaluation, axis=1)
    best_round = np.argsort(mean_eval)[0]
    search_value = mean_eval[best_round]

    ###############################################
    # Update Best Score and Return Negative Value
    # In order to minimize error, instead of maximizing accuracy
    ###############################################
    print(' Stopped After {} Epochs... Validation RMSLE: {:.6f} +- {:.6f}'.format(
        best_round, search_value, std_eval[best_round]
    ))

    return -search_value



In [25]:
bayes_opt = BayesianOptimization(search_node, BAYESIAN_OPTIMIZATION_BOUNDARIES)
bayes_opt.explore(BAYESIAN_OPTIMIZATION_INITIAL_SEARCH_POINTS)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    bayes_opt.maximize(**BAYESIAN_OPTIMIZATION_MAXIMIZE_PARAMS)

AttributeError: 'BayesianOptimization' object has no attribute 'explore'

In [40]:
from sklearn.model_selection import cross_val_score
import numpy as np

input_dataset = Path("../data/quantile_df.csv")
df = pd.read_csv(input_dataset)
df = df.drop(["x","y"],axis=1)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(df.mean())

y = df['target'] 
X = df.drop("target",axis=1)

def get_estimator_score(learning_rate):    
    estimator = XGBRegressor(objective='reg:squarederror', learning_rate=learning_rate)
    scores = cross_val_score(estimator, X, y, scoring='r2', cv=3)
    return scores.mean()

optimizer = BayesianOptimization(
    f=get_estimator_score,
    pbounds={"learning_rate": (0.1, 10)},
    random_state=1,
)
optimizer.maximize(
    init_points=2,
    n_iter=10
)
print("Final result:", optimizer.max)

|   iter    |  target   | learni... |
-------------------------------------


/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (4.228517846555483,)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 88, in __call__
    *args, **kwargs)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 243, in _score
    **self._kwargs)


|  1        |  nan      |  4.229    |


/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (7.231212485077365,)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 88, in __call__
    *args, **kwargs)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 243, in _score
    **self._kwargs)


|  2        |  nan      |  7.231    |


/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (7.231212485077365,)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 88, in __call__
    *args, **kwargs)
  File "/home/547/sg4953/uncoverml/venv/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 243, in _score
    **self._kwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').